In [1]:
import pyedflib 
import numpy as np 
import sagemaker.s3 
import boto3 
import os 
import pandas as pd 
import scipy.signal 
import subprocess

In [2]:
session = sagemaker.Session()
sts_client = session.boto_session.client("sts")
role = session.get_caller_identity_arn()
if "role" in role:
    credentials = session.boto_session.get_credentials()
    credentials = dict(SessionToken=credentials.token, AccessKeyId=credentials.access_key, SecretAccessKey=credentials.secret_key)
else:
    sts_client = session.boto_session.client("sts") 
    assumed_role_object = sts_client.assume_role(
        RoleArn="arn:aws:iam::789873108456:role/DataScientist",
        RoleSessionName="LabelStudioUploadSession",
        DurationSeconds=3600*12
    )
    credentials = assumed_role_object['Credentials']
s3 = boto3.resource(
    's3',
    aws_access_key_id=credentials['AccessKeyId'],
    aws_secret_access_key=credentials['SecretAccessKey'],
    aws_session_token=credentials['SessionToken'],
)



In [54]:
def createFolder(bucket_nam,directory_nam):
    s3 = boto3.client('s3')
    bucket_name = bucket_nam
    directory_name = directory_nam #it's name of your folders
    s3.put_object(Bucket=bucket_name, Key=(directory_name+'/'))

In [55]:
def moveFileInSameBucket(source,target):
    '''
    source : S3 URI
    target : S3 URI
    '''
    cmd = 'aws s3 mv '+source+' '+target
    proc=subprocess.Popen(cmd, shell=True, stdout = subprocess.PIPE) 

In [53]:

def moveFileInSameBucket(source,target):
    '''
    source : S3 URI
    target : S3 URI
    
    '''
    cmd = 'aws s3 mv '+source+' '+target
    proc=subprocess.Popen(cmd, shell=True, stdout = subprocess.PIPE) 

#cmd='aws s3 mv s3://tmp-datascience-apneal/raf21db/edf_input_nox+apneal/g1_nox+apneal.edf s3://tmp-datascience-apneal/raf21db/edf_input_nox+apneal/g1_nox+apneal/g1_nox+apneal.edf '


In [31]:
def downloadDirectoryFroms3(bucketName, remoteDirectoryName):
    s3_resource = boto3.resource(
    's3',
    aws_access_key_id=credentials['AccessKeyId'],
    aws_secret_access_key=credentials['SecretAccessKey'],
    aws_session_token=credentials['SessionToken'],
)
    bucket = s3_resource.Bucket(bucketName) 
    for obj in bucket.objects.filter(Prefix = remoteDirectoryName):
        if not os.path.exists(os.path.dirname(obj.key)):
            os.makedirs(os.path.dirname(obj.key))
        bucket.download_file(obj.key, obj.key) # save to same path